# News Summarization using T5

In [1]:
import pandas as pd
import numpy as np
import datasets
from datasets import Dataset, DatasetDict
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

/opt/homebrew/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9A4710B9-0DA3-36BB-9129-645F282E64B2> /opt/homebrew/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <7856C0E5-3D52-39C7-8515-71217150BD2E> /opt/homebrew/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
df_train1 = pd.read_csv('train.csv')
df_train1.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [3]:
# Drop the 'id' column from the dataframe df_train1 to create a new dataframe df_train
df_train = df_train1.drop(columns=['id'])

# Select the first 1000 rows of the dataframe df_train
df_train = df_train.iloc[:1000, :]

# Define a prefix string to be added to each article
prefix = 'summarize: '

# Add the prefix to each article in the 'article' column
df_train['article'] = prefix + df_train['article']

# Display the first few rows of the modified dataframe df_train
df_train.head()


,article,highlights
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: (CNN) -- Ralph Mata was an internal...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: (CNN) -- With a breezy sweep of his...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...


In [4]:
# Loading the testing dataset
df_test = pd.read_csv('test.csv')
df_test = df_test.drop(columns=['id'])

In [5]:
# Loading the validation dataset
df_val = pd.read_csv('validation.csv')

# Select the first 200 rows of the dataframe df_val
df_val = df_val.iloc[:200, :]

# Drop the 'id' column from the dataframe
df_val = df_val.drop(columns=['id'])
df_val.head()

,article,highlights
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [6]:
print("train and val shape:", df_train.shape, "test shape:",df_val.shape)

train and val shape: (1000, 2) test shape: (200, 2)


In [7]:
model_name = "t5-base"
# Load the T5 tokenizer and model for summarization
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("mps")

# Create a summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Extract model parameters
def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total_params, trainable_params = get_model_parameters(model)
print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Total Parameters: 222903552
Trainable Parameters: 222903552


In [8]:
# Converting the pandas dataset to huggingface dataset
# save for every model inference
global_train_df = df_train
global_test_df = df_val
train_df = Dataset.from_pandas(df_train)
val_df = Dataset.from_pandas(df_val)

In [9]:
# Fitting into dataset dict
train_val_test_dataset = DatasetDict({
    'train': train_df,
    'val': val_df})

print(type(train_val_test_dataset))
train_val_test_dataset

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 1000
    })
    val: Dataset({
        features: ['article', 'highlights'],
        num_rows: 200
    })
})

## T5 fine-tuning

In [10]:
def prepare_dataset(data):
    inputs = data["article"]

    # Tokenize the inputs using the tokenizer
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Tokenize the 'highlights' column from the data to be used as labels
    labels = tokenizer(text_target=data["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_data = train_val_test_dataset.map(prepare_dataset, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
tokenized_data["train"]

Dataset({
    features: ['article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [13]:
# Padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [14]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return result

In [15]:
# set up hyper-parameters
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-news",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    #fp16=False,
    report_to="none",
    #use_mps_device = True,
    no_cuda=True
)

/opt/homebrew/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/transformers/training_args.py:1509: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [16]:
# setup trainer
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data["train"],
    eval_dataset = tokenized_data["val"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

In [17]:
trainer.train()

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 2.000952959060669, 'eval_rouge1': 0.4096947183157629, 'eval_rouge2': 0.18412682560807386, 'eval_rougeL': 0.27328271668175974, 'eval_rougeLsum': 0.27402598971751746, 'eval_runtime': 2382.1671, 'eval_samples_per_second': 0.084, 'eval_steps_per_second': 0.005, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 1.792526125907898, 'eval_rouge1': 0.4165574951635077, 'eval_rouge2': 0.18822704232449258, 'eval_rougeL': 0.2778257163324279, 'eval_rougeLsum': 0.2780420773732868, 'eval_runtime': 1112.4833, 'eval_samples_per_second': 0.18, 'eval_steps_per_second': 0.012, 'epoch': 2.0}


...

{'eval_loss': 1.7263814210891724, 'eval_rouge1': 0.41932816799419226, 'eval_rouge2': 0.19330916064919734, 'eval_rougeL': 0.2798923563286625, 'eval_rougeLsum': 0.2808034825606014, 'eval_runtime': 1125.7201, 'eval_samples_per_second': 0.178, 'eval_steps_per_second': 0.012, 'epoch': 3.0}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'min_length': 30, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 1.7183984518051147, 'eval_rouge1': 0.415850055053714, 'eval_rouge2': 0.1916597483141818, 'eval_rougeL': 0.27713887637781587, 'eval_rougeLsum': 0.2777391014799023, 'eval_runtime': 1143.3929, 'eval_samples_per_second': 0.175, 'eval_steps_per_second': 0.011, 'epoch': 4.0}
{'train_runtime': 13601.4345, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.019, 'train_loss': 1.7056186009967138, 'epoch': 4.0}


TrainOutput(global_step=252, training_loss=1.7056186009967138, metrics={'train_runtime': 13601.4345, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.019, 'total_flos': 2435831562240000.0, 'train_loss': 1.7056186009967138, 'epoch': 4.0})

In [18]:
# save the model
model_path = "t5-news"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'min_length': 30, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


('t5-news/tokenizer_config.json',
 't5-news/special_tokens_map.json',
 't5-news/spiece.model',
 't5-news/added_tokens.json',
 't5-news/tokenizer.json')